# x402 Fee Facilitator Demo (Local Testing)

Test the **fee-based x402 facilitator** locally. No whitelist, no splitter contract — just a simple fee model:

## 💰 Fee Flow

1. **Buyer** signs EIP-3009 authorization: `transferWithAuthorization(buyer → seller, amount)`
2. **Facilitator** verifies signature + checks seller's USDC allowance for fee
3. **Facilitator** settles: executes `transferWithAuthorization` on-chain (buyer → seller)
4. **Facilitator** collects fee: `transferFrom(seller → facilitator, feeAmount)` post-settlement

## 🔑 Key Insight

The **seller (merchant)** pays the fee, not the buyer! The seller must pre-approve USDC spending by the facilitator wallet. This is a one-time `approve()` call on the USDC contract.

## 📋 Prerequisites

1. Start the local facilitator: `cd x402_facilitator && npm run dev`
2. Set env vars in `notebooks/.env`:
   - `TEST_WALLET_PRIVATE_KEY` — Buyer's private key (has USDC on Sepolia)
   - `NFT_WALLET_PUBLIC_KEY` — Seller's address (receives payment)
   - `NFT_WALLET_PRIVATE_KEY` — Seller's private key (for USDC approval)

In [ ]:
// Setup: Imports and Configuration
import { load } from "https://deno.land/std@0.224.0/dotenv/mod.ts";
import { privateKeyToAccount } from "npm:viem@2/accounts";
import { createPublicClient, createWalletClient, http, formatUnits, getContract } from "npm:viem@2";
import { optimism, optimismSepolia } from "npm:viem@2/chains";

// Load environment variables
const env = await load({ export: true });

const BUYER_PRIVATE_KEY = env.TEST_WALLET_PRIVATE_KEY;
const SELLER_ADDRESS = env.NFT_WALLET_PUBLIC_KEY as `0x${string}`;
const SELLER_PRIVATE_KEY = env.NFT_WALLET_PRIVATE_KEY;

// Create buyer account
const buyerAccount = privateKeyToAccount(`0x${BUYER_PRIVATE_KEY}`);
const sellerAccount = privateKeyToAccount(`0x${SELLER_PRIVATE_KEY}`);

console.log("🚀 x402 Fee Facilitator Demo (Local)");
console.log(`   Buyer:  ${buyerAccount.address}`);
console.log(`   Seller: ${SELLER_ADDRESS}`);
console.log(`   Seller (from key): ${sellerAccount.address}`);

## Network & Payment Configuration

Choose between **Testnet** (Optimism Sepolia) and **Mainnet** (real money!).

The buyer pays `PAYMENT_AMOUNT` to the seller. The facilitator fee (0.01 USDC) is collected **from the seller** after settlement — the buyer pays nothing extra.

In [ ]:
// ⚠️ NETWORK SELECTION
const USE_MAINNET = false;

const config = USE_MAINNET ? {
    chain: optimism,
    chainId: 10,
    caip2Network: "eip155:10" as const,
    networkName: "Optimism Mainnet",
    usdcAddress: "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85" as `0x${string}`,
    usdcName: "USD Coin",
    rpcUrl: "https://mainnet.optimism.io",
} : {
    chain: optimismSepolia,
    chainId: 11155420,
    caip2Network: "eip155:11155420" as const,
    networkName: "Optimism Sepolia (Testnet)",
    usdcAddress: "0x5fd84259d66Cd46123540766Be93DFE6D43130D7" as `0x${string}`,
    usdcName: "USDC",
    rpcUrl: "https://sepolia.optimism.io",
};

// What the buyer pays (seller receives this minus nothing — fee is separate!)
const PAYMENT_AMOUNT = "20000";  // $0.02 USDC (6 decimals)

if (USE_MAINNET) {
    console.log(`\n🚨 WARNING: Using REAL MONEY on ${config.networkName}!`);
} else {
    console.log(`\n🧪 Using testnet: ${config.networkName}`);
}
console.log(`   Chain ID: ${config.chainId}`);
console.log(`   USDC: ${config.usdcAddress}`);
console.log(`   Payment: ${PAYMENT_AMOUNT} (${Number(PAYMENT_AMOUNT) / 1e6} USDC)`);

## Step 1: Query /supported

Check what the facilitator supports. The `facilitator_fee` extension shows the fee amount and the facilitator's wallet address (needed for USDC approval).

In [ ]:
// Facilitator endpoint (local dev server)
const FACILITATOR_URL = "http://localhost:8080";

const SUPPORTED_URL = `${FACILITATOR_URL}/supported`;
const VERIFY_URL = `${FACILITATOR_URL}/verify`;
const SETTLE_URL = `${FACILITATOR_URL}/settle`;

// Query /supported
const supportedResponse = await fetch(SUPPORTED_URL);
const supported = await supportedResponse.json();

console.log(`📡 GET ${SUPPORTED_URL} → ${supportedResponse.status}`);

// Show supported networks
console.log(`\n🌐 Supported Networks:`);
for (const kind of supported.kinds || []) {
    console.log(`   - ${kind.network} (scheme: ${kind.scheme}, x402v${kind.x402Version})`);
}

// Extract fee extension
const feeExtension = supported.extensions?.find((e: any) => e.name === "facilitator_fee");
let facilitatorAddress: string | null = null;
let feeAmount: string | null = null;

if (feeExtension) {
    facilitatorAddress = feeExtension.fee?.recipient;
    feeAmount = feeExtension.fee?.amount;
    console.log(`\n💸 Fee Extension:`);
    console.log(`   Fee: ${feeExtension.fee?.description}`);
    console.log(`   Amount: ${feeAmount} (${Number(feeAmount) / 1e6} USDC)`);
    console.log(`   Facilitator Address: ${facilitatorAddress}`);
    console.log(`   Collection: ${feeExtension.fee?.collection}`);
    console.log(`\n🔧 Setup Required:`);
    console.log(`   ${feeExtension.setup?.description}`);
    console.log(`   Function: ${feeExtension.setup?.function}`);
    console.log(`   Spender: ${feeExtension.setup?.spender}`);
} else {
    console.log(`\n⚠️ No facilitator_fee extension found — fees may be disabled`);
}

// Show signers
if (supported.signers?.["eip155:*"]) {
    console.log(`\n📝 Facilitator Signers: ${supported.signers["eip155:*"].join(", ")}`);
}

## Step 2: Check USDC Balances (Before)

We check the USDC balances of all three parties before any transaction:
- **Buyer** — will pay the full `PAYMENT_AMOUNT`
- **Seller** — will receive `PAYMENT_AMOUNT`, then the facilitator takes the fee via `transferFrom`
- **Facilitator** — collects the fee after settlement

In [ ]:
// Minimal ERC-20 ABI for balance, allowance, and approval
const ERC20_ABI = [
    {
        name: "balanceOf",
        type: "function",
        stateMutability: "view",
        inputs: [{ name: "account", type: "address" }],
        outputs: [{ name: "", type: "uint256" }],
    },
    {
        name: "allowance",
        type: "function",
        stateMutability: "view",
        inputs: [
            { name: "owner", type: "address" },
            { name: "spender", type: "address" },
        ],
        outputs: [{ name: "", type: "uint256" }],
    },
    {
        name: "approve",
        type: "function",
        stateMutability: "nonpayable",
        inputs: [
            { name: "spender", type: "address" },
            { name: "value", type: "uint256" },
        ],
        outputs: [{ name: "", type: "bool" }],
    },
] as const;

// Create public client for reading on-chain state
const publicClient = createPublicClient({
    chain: chainConfig,
    transport: http(chainConfig.rpcUrls.default.http[0]),
});

// Helper to read USDC balance
async function getUsdcBalance(address: `0x${string}`): Promise<bigint> {
    return publicClient.readContract({
        address: usdcAddress as `0x${string}`,
        abi: ERC20_ABI,
        functionName: "balanceOf",
        args: [address],
    });
}

// Check all balances
const balancesBefore = {
    buyer: await getUsdcBalance(buyerAccount.address),
    seller: await getUsdcBalance(SELLER_ADDRESS as `0x${string}`),
    facilitator: facilitatorAddress
        ? await getUsdcBalance(facilitatorAddress as `0x${string}`)
        : 0n,
};

console.log(`💰 USDC Balances Before:`);
console.log(`   Buyer     (${buyerAccount.address}): ${Number(balancesBefore.buyer) / 1e6} USDC`);
console.log(`   Seller    (${SELLER_ADDRESS}): ${Number(balancesBefore.seller) / 1e6} USDC`);
if (facilitatorAddress) {
    console.log(`   Facilitator (${facilitatorAddress}): ${Number(balancesBefore.facilitator) / 1e6} USDC`);
}

// Check if buyer has enough
const paymentBigInt = BigInt(PAYMENT_AMOUNT);
if (balancesBefore.buyer < paymentBigInt) {
    console.log(`\n⚠️ Buyer needs at least ${Number(paymentBigInt) / 1e6} USDC but only has ${Number(balancesBefore.buyer) / 1e6}`);
} else {
    console.log(`\n✅ Buyer has enough USDC for payment`);
}

## Step 3: Seller Approves USDC for Fee Collection

The fee model requires the **seller** to have an active ERC-20 `approve` for the facilitator's address.
This is a **one-time setup** per seller — the approval amount covers many settlements.

**Flow:**
1. Seller calls `USDC.approve(facilitatorAddress, amount)` — e.g. 100 USDC = 10,000 settlements at 0.01 USDC fee
2. The facilitator checks `allowance(seller, facilitator)` during `/verify`
3. After `/settle`, the facilitator calls `transferFrom(seller, facilitator, fee)` to collect

If the allowance is already sufficient, you can skip the approval transaction.

In [ ]:
// First check existing allowance
const sellerAddr = SELLER_ADDRESS as `0x${string}`;
const facilitatorAddr = facilitatorAddress as `0x${string}`;

const currentAllowance = await publicClient.readContract({
    address: usdcAddress as `0x${string}`,
    abi: ERC20_ABI,
    functionName: "allowance",
    args: [sellerAddr, facilitatorAddr],
});

const feeAmountBigInt = feeAmount ? BigInt(feeAmount) : 10000n;
const feePerSettlement = Number(feeAmountBigInt) / 1e6;
const remainingSettlements = feeAmountBigInt > 0n
    ? Number(currentAllowance / feeAmountBigInt)
    : Infinity;

console.log(`🔍 Current Seller Allowance:`);
console.log(`   Allowance: ${Number(currentAllowance) / 1e6} USDC`);
console.log(`   Fee per settlement: ${feePerSettlement} USDC`);
console.log(`   Remaining settlements: ${remainingSettlements}`);

if (currentAllowance >= feeAmountBigInt) {
    console.log(`\n✅ Allowance is sufficient for at least 1 settlement`);
    console.log(`   (skip approval step if you want)`);
} else {
    console.log(`\n⚠️ Allowance insufficient — approval required!`);
}

In [ ]:
// Approve USDC spending for the facilitator
// Amount: 100 USDC = covers 10,000 settlements at 0.01 USDC fee
const APPROVAL_AMOUNT = 100_000_000n; // 100 USDC (6 decimals)

// Create seller wallet client for the approval transaction
const sellerWalletClient = createWalletClient({
    account: sellerAccount,
    chain: chainConfig,
    transport: http(chainConfig.rpcUrls.default.http[0]),
});

console.log(`📝 Submitting approve(${facilitatorAddr}, ${Number(APPROVAL_AMOUNT) / 1e6} USDC)...`);

const approveTxHash = await sellerWalletClient.writeContract({
    address: usdcAddress as `0x${string}`,
    abi: ERC20_ABI,
    functionName: "approve",
    args: [facilitatorAddr, APPROVAL_AMOUNT],
});

console.log(`   Tx Hash: ${approveTxHash}`);
console.log(`   Waiting for confirmation...`);

const approveReceipt = await publicClient.waitForTransactionReceipt({
    hash: approveTxHash,
});

console.log(`   ✅ Approval confirmed in block ${approveReceipt.blockNumber}`);
console.log(`   Gas used: ${approveReceipt.gasUsed}`);

// Verify new allowance
const newAllowance = await publicClient.readContract({
    address: usdcAddress as `0x${string}`,
    abi: ERC20_ABI,
    functionName: "allowance",
    args: [sellerAddr, facilitatorAddr],
});

const newRemainingSettlements = feeAmountBigInt > 0n
    ? Number(newAllowance / feeAmountBigInt)
    : Infinity;

console.log(`\n💰 Updated Allowance:`);
console.log(`   Allowance: ${Number(newAllowance) / 1e6} USDC`);
console.log(`   Settlements available: ${newRemainingSettlements}`);

## Step 4: Create x402 Payment Payload

Now we create a standard x402 payment from the buyer to the seller using the `exact` scheme.
The buyer signs an EIP-3009 `transferWithAuthorization` that moves USDC from buyer → seller.

The facilitator fee is **not** part of this payload — it's collected separately after settlement.

In [ ]:
// Import x402 packages for payment creation
import { x402Client } from "npm:@x402/fetch@^2.0.0";
import type { PaymentRequirements, PaymentPayload, Network } from "npm:@x402/core@^2.0.0";
import { ExactEvmScheme } from "npm:@x402/evm@^2.0.0";

// Create x402 client with standard ExactEvmScheme (NOT splitter!)
const evmScheme = new ExactEvmScheme(buyerAccount);
const client = new x402Client();
client.register("eip155:*" as Network, evmScheme);

// Build PaymentRequirements — standard x402 exact scheme
// The seller simply receives the payment amount
const paymentRequirements: PaymentRequirements = {
    scheme: "exact",                          // ← Standard scheme!
    network: chainConfig.id === 10 ? "eip155:10" : "eip155:11155420",
    amount: PAYMENT_AMOUNT,                   // What the buyer pays
    asset: usdcAddress,
    payTo: SELLER_ADDRESS as `0x${string}`,   // Seller's own address
    maxTimeoutSeconds: 3600,
    extra: {
        name: chainConfig.id === 10 ? "USD Coin" : "USDC",
        version: "2"
    }
};

// Build mock 402 response (normally comes from server)
const paymentRequired = {
    x402Version: 2,
    accepts: [paymentRequirements],
    resource: {
        url: "https://example.com/resource",
        description: "Test payment with fee facilitator",
        mimeType: "application/json"
    },
    extensions: {}
};

// Create payment payload — buyer signs EIP-3009 transferWithAuthorization
const paymentPayload: PaymentPayload = await client.createPaymentPayload(paymentRequired as any);

console.log("✅ Payment Payload created (standard exact scheme)");
console.log(`   Scheme: exact (standard — no splitter)`);
console.log(`   From: ${paymentPayload.payload?.authorization?.from}`);
console.log(`   To (Seller): ${paymentPayload.payload?.authorization?.to}`);
console.log(`   Amount: ${paymentPayload.payload?.authorization?.value} (${Number(paymentPayload.payload?.authorization?.value || 0) / 1e6} USDC)`);
console.log(`\n💡 Key difference from splitter:`);
console.log(`   • Payment goes DIRECTLY to seller (not via contract)`);
console.log(`   • Fee is collected AFTER settlement via transferFrom`);

## Step 5: Verify Payment

Send the payment to `/verify`. The facilitator will:
1. Validate the EIP-3009 signature
2. Check buyer has sufficient USDC balance
3. **Check seller's USDC allowance for fee collection** (via `onAfterVerify` hook)

If the seller hasn't approved USDC spending, the verify response will include `invalidReason: "insufficient_fee_allowance"` with details about the required approval.

In [ ]:
// Build verify request
const verifyRequest = {
    paymentPayload: paymentPayload,
    paymentRequirements: paymentRequirements
};

console.log("🔍 Sending verification request...");

const verifyResponse = await fetch(VERIFY_URL, {
    method: "POST",
    headers: { "Content-Type": "application/json" },
    body: JSON.stringify(verifyRequest)
});

const verifyResult = await verifyResponse.json();

console.log(`\n📦 Verify Response (Status ${verifyResponse.status}):`);
console.log(JSON.stringify(verifyResult, null, 2));

if (verifyResult.isValid) {
    console.log(`\n✅ Payment is VALID!`);
    console.log(`   Payer: ${verifyResult.payer}`);
    if (verifyResult.feeRequired !== undefined) {
        console.log(`   Fee Required: ${verifyResult.feeRequired}`);
    }
    if (verifyResult.recipient) {
        console.log(`   Fee Recipient (Seller): ${verifyResult.recipient}`);
    }
} else {
    console.log(`\n❌ Payment is INVALID!`);
    console.log(`   Reason: ${verifyResult.invalidReason}`);
    if (verifyResult.invalidReason === "insufficient_fee_allowance") {
        console.log(`\n💡 Fix: Run the approval cell above (Step 3)`);
        console.log(`   Required allowance: ${verifyResult.requiredAllowance}`);
        console.log(`   Current allowance: ${verifyResult.currentAllowance}`);
        console.log(`   Facilitator address: ${verifyResult.facilitatorAddress}`);
    }
}

## Step 6: Settle Payment

Execute the settlement:
1. Facilitator calls `transferWithAuthorization` on USDC (buyer → seller)
2. **If fee is required:** Facilitator calls `transferFrom(seller, facilitator, feeAmount)` on USDC

The settle response will include a `fee` object showing whether the fee was collected.

In [ ]:
console.log(`💸 Attempting Settlement...`);
console.log(`   Network: ${paymentRequirements.network}`);
console.log(`   Buyer pays: ${Number(PAYMENT_AMOUNT) / 1e6} USDC → Seller`);
console.log(`   Fee: ${Number(feeAmountBigInt) / 1e6} USDC (collected after settlement)`);

if (USE_MAINNET) {
    console.log(`\n🚨 WARNING: REAL transaction with REAL MONEY!`);
}

const settleResponse = await fetch(SETTLE_URL, {
    method: "POST",
    headers: { "Content-Type": "application/json" },
    body: JSON.stringify(verifyRequest)
});

const settleResult = await settleResponse.json();

console.log(`\n📦 Settle Response (Status ${settleResponse.status}):`);
console.log(JSON.stringify(settleResult, null, 2));

if (settleResult.success) {
    console.log(`\n🎉 Settlement successful!`);
    console.log(`   Transaction: ${settleResult.transaction}`);
    if (settleResult.network) {
        console.log(`   Network: ${settleResult.network}`);
    }
    // Fee details
    if (settleResult.fee) {
        console.log(`\n💰 Fee Collection:`);
        console.log(`   Collected: ${settleResult.fee.collected}`);
        if (settleResult.fee.txHash) {
            console.log(`   Fee Tx: ${settleResult.fee.txHash}`);
        }
        if (settleResult.fee.error) {
            console.log(`   ⚠️ Fee Error: ${settleResult.fee.error}`);
        }
    }
} else {
    console.log(`\n❌ Settlement failed`);
    console.log(`   Reason: ${settleResult.errorReason || "unknown"}`);
}

## Step 7: Post-Settlement Verification

Check balances and allowance after settlement to verify the complete flow:
- **Buyer** should have paid `PAYMENT_AMOUNT`
- **Seller** should have received `PAYMENT_AMOUNT - fee`
- **Facilitator** should have gained `fee`
- **Allowance** should have decreased by `fee`

In [ ]:
// Check USDC balances AFTER settlement
const balancesAfter = {
    buyer: await getUsdcBalance(buyerAccount.address),
    seller: await getUsdcBalance(SELLER_ADDRESS as `0x${string}`),
    facilitator: facilitatorAddress
        ? await getUsdcBalance(facilitatorAddress as `0x${string}`)
        : 0n,
};

// Check allowance AFTER settlement
const allowanceAfter = await publicClient.readContract({
    address: usdcAddress as `0x${string}`,
    abi: ERC20_ABI,
    functionName: "allowance",
    args: [sellerAddr, facilitatorAddr],
});

console.log(`💰 USDC Balances AFTER Settlement:`);
console.log(`   Buyer:       ${Number(balancesAfter.buyer) / 1e6} USDC`);
console.log(`   Seller:      ${Number(balancesAfter.seller) / 1e6} USDC`);
if (facilitatorAddress) {
    console.log(`   Facilitator: ${Number(balancesAfter.facilitator) / 1e6} USDC`);
}

// Calculate deltas
const buyerDelta = balancesBefore.buyer - balancesAfter.buyer;
const sellerDelta = balancesAfter.seller - balancesBefore.seller;
const facilitatorDelta = balancesAfter.facilitator - balancesBefore.facilitator;
const allowanceDelta = currentAllowance - allowanceAfter;

console.log(`\n📊 Balance Changes:`);
console.log(`   Buyer spent:          ${Number(buyerDelta) / 1e6} USDC`);
console.log(`   Seller gained:        ${Number(sellerDelta) / 1e6} USDC`);
console.log(`   Facilitator gained:   ${Number(facilitatorDelta) / 1e6} USDC`);
console.log(`   Allowance decreased:  ${Number(allowanceDelta) / 1e6} USDC`);

console.log(`\n🔍 Allowance Status:`);
console.log(`   Before: ${Number(currentAllowance) / 1e6} USDC`);
console.log(`   After:  ${Number(allowanceAfter) / 1e6} USDC`);
const settlementsRemaining = feeAmountBigInt > 0n ? Number(allowanceAfter / feeAmountBigInt) : Infinity;
console.log(`   Settlements remaining: ${settlementsRemaining}`);

// Verify correctness
console.log(`\n✅ Verification:`);

// Buyer should have paid PAYMENT_AMOUNT
if (buyerDelta === BigInt(PAYMENT_AMOUNT)) {
    console.log(`   ✅ Buyer paid correct amount: ${Number(buyerDelta) / 1e6} USDC`);
} else {
    console.log(`   ❌ Buyer payment mismatch! Expected: ${Number(PAYMENT_AMOUNT) / 1e6}, Got: ${Number(buyerDelta) / 1e6}`);
}

// Seller should have gained PAYMENT_AMOUNT - fee
const expectedSellerGain = BigInt(PAYMENT_AMOUNT) - feeAmountBigInt;
if (sellerDelta === expectedSellerGain) {
    console.log(`   ✅ Seller received correct net: ${Number(sellerDelta) / 1e6} USDC (payment - fee)`);
} else {
    console.log(`   ❌ Seller amount mismatch! Expected: ${Number(expectedSellerGain) / 1e6}, Got: ${Number(sellerDelta) / 1e6}`);
}

// Facilitator should have gained fee
if (facilitatorDelta === feeAmountBigInt) {
    console.log(`   ✅ Facilitator collected correct fee: ${Number(facilitatorDelta) / 1e6} USDC`);
} else {
    console.log(`   ❌ Fee mismatch! Expected: ${Number(feeAmountBigInt) / 1e6}, Got: ${Number(facilitatorDelta) / 1e6}`);
}

// Allowance should have decreased by fee
if (allowanceDelta === feeAmountBigInt) {
    console.log(`   ✅ Allowance decreased by fee amount: ${Number(allowanceDelta) / 1e6} USDC`);
} else {
    console.log(`   ❌ Allowance mismatch! Expected decrease: ${Number(feeAmountBigInt) / 1e6}, Got: ${Number(allowanceDelta) / 1e6}`);
}

// Conservation of funds
if (sellerDelta + facilitatorDelta === buyerDelta) {
    console.log(`   ✅ Conservation of funds: Seller + Fee = Buyer spent`);
} else {
    console.log(`   ❌ Funds don't add up!`);
}

## Step 8: Block Explorer

View the transactions on the block explorer.

In [ ]:
// Block Explorer URLs
const networkToExplorer: Record<string, string> = {
    "eip155:10": "https://optimistic.etherscan.io/tx/",
    "eip155:11155420": "https://sepolia-optimism.etherscan.io/tx/",
};

const network = paymentRequirements.network;
const explorerBase = networkToExplorer[network] || "";

if (settleResult.success && settleResult.transaction) {
    const settleTxUrl = explorerBase ? `${explorerBase}${settleResult.transaction}` : settleResult.transaction;
    console.log(`🔍 Settlement Transaction:`);
    console.log(`   ${settleTxUrl}`);

    if (settleResult.fee?.txHash) {
        const feeTxUrl = explorerBase ? `${explorerBase}${settleResult.fee.txHash}` : settleResult.fee.txHash;
        console.log(`\n💰 Fee Collection Transaction:`);
        console.log(`   ${feeTxUrl}`);
    }
    
    console.log(`\n📊 Summary:`);
    console.log(`   Network: ${network}`);
    console.log(`   USDC: ${usdcAddress}`);
    console.log(`   Buyer → Seller: ${Number(PAYMENT_AMOUNT) / 1e6} USDC`);
    console.log(`   Seller → Facilitator: ${Number(feeAmountBigInt) / 1e6} USDC (fee)`);
    console.log(`   Net to Seller: ${(Number(PAYMENT_AMOUNT) - Number(feeAmountBigInt)) / 1e6} USDC`);
} else {
    console.log(`⚠️ No transaction available — settlement may have failed`);
}

## Summary

This notebook tests the **fee-based x402 facilitator** (without splitter contract):

| Step | What happens |
|------|-------------|
| 1. `/supported` | Returns `facilitator_fee` extension with fee amount & facilitator address |
| 2. Balance check | USDC balances of buyer, seller, facilitator |
| 3. Seller approval | One-time `USDC.approve(facilitator, amount)` — enables fee collection |
| 4. Payment creation | Standard `exact` scheme — buyer signs EIP-3009 to seller |
| 5. `/verify` | Validates signature + checks seller's allowance for fee |
| 6. `/settle` | Executes payment on-chain, then collects fee via `transferFrom` |
| 7. Verification | Checks all balances and allowance changed correctly |

### Key Differences from Splitter Facilitator

| Feature | Splitter | Fee Facilitator |
|---------|----------|-----------------|
| Scheme | `exact-split` (custom) | `exact` (standard) ✅ |
| Payment target | Splitter contract | Seller directly ✅ |
| Fee collection | On-chain split in contract | Post-settlement `transferFrom` |
| Seller setup | None | One-time USDC `approve` |
| Complexity | Custom contract + scheme | Standard x402 + ERC-20 approval ✅ |

### Prerequisites
- Facilitator running locally: `cd x402_facilitator && npm run dev`
- `.env` in `notebooks/` with `TEST_WALLET_PRIVATE_KEY`, `NFT_WALLET_PRIVATE_KEY`, `NFT_WALLET_PUBLIC_KEY`
- Buyer and seller wallets funded with USDC on Optimism Sepolia